# Alphalens Analysis

### Loading Libraries

In [2]:
# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd

# Path
from pathlib import Path

# Alphalens
from alphalens.tears import create_summary_tear_sheet
from alphalens.utils import get_clean_factor_and_forward_returns

# Warnings
import warnings

### Loading Data

In [3]:
with pd.HDFStore('data.h5') as store:
    lr_predictions = store['lr/predictions']
    lasso_predictions = store['lasso/predictions']
    lasso_scores = store['lasso/scores']
    ridge_predictions = store['ridge/predictions']
    ridge_scores = store['ridge/scores']

In [4]:
DATA_STORE = Path('..', 'data', 'assets.h5')

In [8]:
def get_factor(predictions):
    return (predictions.unstack('symbol')
            .dropna(how='all')
            .stack()
            .tz_localize('UTC', level='date')
            .sort_index())    

In [9]:
def get_best_alpha(scores):
    return scores.groupby('alpha').ic.mean().idxmax()

In [10]:
def get_factor(predictions):
    return (predictions.unstack('symbol')
            .dropna(how='all')
            .stack()
            .tz_localize('UTC', level='date')
            .sort_index())    

### Linear Regression

In [11]:
lr_factor = get_factor(lr_predictions.predicted.swaplevel())

lr_factor.head()

In [12]:
tickers = lr_factor.index.get_level_values('symbol').unique()

In [13]:
trade_prices = get_trade_prices(tickers, 2014, 2017)

trade_prices.info()

In [14]:
lr_factor_data = get_clean_factor_and_forward_returns(factor=lr_factor,
                                                      prices=trade_prices,
                                                      quantiles=5,
                                                      periods=(1, 5, 10, 21))

lr_factor_data.info()

In [15]:
create_summary_tear_sheet(lr_factor_data);

### Ridge Regression

In [16]:
best_ridge_alpha = get_best_alpha(ridge_scores)

ridge_predictions = ridge_predictions[ridge_predictions.alpha==best_ridge_alpha].drop('alpha', axis=1)

In [17]:
ridge_factor = get_factor(ridge_predictions.predicted.swaplevel())

ridge_factor.head()

In [18]:
ridge_factor_data = get_clean_factor_and_forward_returns(factor=ridge_factor,
                                                         prices=trade_prices,
                                                         quantiles=5,
                                                         periods=(1, 5, 10, 21))

ridge_factor_data.info()

In [19]:
create_summary_tear_sheet(ridge_factor_data);

### Lasso Regression

In [20]:
best_lasso_alpha = get_best_alpha(lasso_scores)

lasso_predictions = lasso_predictions[lasso_predictions.alpha==best_lasso_alpha].drop('alpha', axis=1)

In [21]:
lasso_factor = get_factor(lasso_predictions.predicted.swaplevel())

lasso_factor.head()

In [22]:
lasso_factor_data = get_clean_factor_and_forward_returns(factor=lasso_factor,
                                                      prices=trade_prices,
                                                      quantiles=5,
                                                      periods=(1, 5, 10, 21))

lasso_factor_data.info()

In [23]:
create_summary_tear_sheet(lasso_factor_data);